# 1. Introducción

### Train: es para entrenar el algoritmo
### Test: es para subir las predicciones

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
ejemplo = pd.read_csv("data/ejemploRespuesta.csv")

In [2]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 10 filas

### El ejemplo de respuesta parece tener una fila como índice

In [3]:
ejemplo

,4941,4200756.25126322
0,51775,1.112322e+06
1,115253,1.377424e+06
2,299321,1.364169e+06
3,173570,1.284638e+06
4,30862,2.411320e+06
5,244471,3.140350e+06
6,127794,1.046047e+06
7,71558,3.670553e+06
8,218011,1.854607e+06
9,148208,2.345045e+06


# 2. Preprocesamiento de los Datos

### Se quita id, titulo, descripción, direccion e Idzona por no entenderlos como tan relevantes

In [4]:
dfinicial = train
dfinicial = dfinicial.drop( columns = ['id', 'titulo', 'descripcion', 'direccion', 'idzona'] )

dftest = test
dftest = dftest.drop( columns = ['id', 'titulo', 'descripcion', 'direccion', 'idzona'] )

### Parseo de valores sin NaN

In [5]:
 def parseoNan(df):
    df['gimnasio'] = df['gimnasio'].astype(np.uint8)
    df['usosmultiples'] = df['usosmultiples'].astype(np.uint8)
    df['piscina'] = df['piscina'].astype(np.uint8)
    df['escuelascercanas'] = df['escuelascercanas'].astype(np.uint8)
    df['centroscomercialescercanos'] = df['centroscomercialescercanos'].astype(np.uint8)

    df['fecha'] = df['fecha'].astype('datetime64')
    
parseoNan(dfinicial)
parseoNan(dftest)

dfinicial['precio'] = dfinicial['precio'].astype(np.uint32)

### Como son pocos casos, se eliminan los que no tienen provincia, ciudad o tipo de propiedad

In [6]:
len( dfinicial[ dfinicial['provincia'].isnull()     ] ) + \
len( dfinicial[ dfinicial['ciudad'].isnull()        ] ) + \
len( dfinicial[ dfinicial['tipodepropiedad'].isnull() ] ) 

573

In [7]:
dfinicial = dfinicial.dropna( subset = ['provincia', 'ciudad', 'tipodepropiedad'] )

### Para el test, no se pueden eliminar esos casos (se debe dar una respuesta), entonces, se llenan con la moda

In [8]:
dftest.loc[ dftest['provincia'].isnull(), 'provincia'] = 'Distrito Federal'
dftest.loc[ dftest['tipodepropiedad'].isnull(), 'tipodepropiedad'] = 'Casa'
dftest.loc[ dftest['ciudad'].isnull(), 'ciudad'] = 'Benito Juárez'

### Como se vio en el tp1, hay muchas coordenadas geográficas faltantes o mal cargadas. Se aprovecha para corregir esto, y quitar Provincia y Ciudad

### Se toman los valores de la latitud y la longitud de cada provincia, y se reemplazan con los actuales, rellenando los NaN

In [9]:
dfinicial['provincia'].unique()

array(['Distrito Federal', 'Jalisco', 'Edo. de México', 'Oaxaca',
       'Quintana Roo', 'Colima', 'San luis Potosí', 'Nuevo León',
       'Querétaro', 'Tamaulipas', 'Puebla', 'Yucatán', 'Morelos',
       'Guerrero', 'Tabasco', 'Guanajuato', 'Hidalgo', 'Veracruz',
       'Chihuahua', 'Aguascalientes', 'Sonora', 'Michoacán',
       'Baja California Norte', 'Baja California Sur', 'Coahuila',
       'Durango', 'Sinaloa', 'Chiapas', 'Nayarit', 'Tlaxcala', 'Campeche',
       'Zacatecas'], dtype=object)

In [10]:
provincias = [ 'Distrito Federal'      ,
            'Jalisco'               ,
            'Edo. de México'      ,
            'Oaxaca'                ,
            'Quintana Roo'          ,
            'Colima'                ,
            'San luis Potosí'       ,
            'Nuevo León'            ,
            'Querétaro'             ,
            'Tamaulipas'            ,
            'Puebla'                ,
            'Yucatán'               ,
            'Morelos'               ,
            'Guanajuato'            ,
            'Guerrero'              ,
            'Tabasco'               ,
            'Hidalgo'               ,
            'Veracruz'              ,
            'Chihuahua'             ,
            'Aguascalientes'        ,
            'Sonora'                ,
            'Michoacán'             ,
            'Baja California Norte' ,
            'Baja California Sur'   ,
            'Coahuila'              ,
            'Durango'               ,
            'Sinaloa'               ,
            'Chiapas'               ,
            'Nayarit'               ,
            'Tlaxcala'              ,
            'Campeche'              ,
            'Zacatecas'             
        ]                       

# Fuente: https://www.coordenadas-gps.com/

latitud = { 'Distrito Federal' : 19.32, 
            'Jalisco' : 20.33,
            'Edo. de México' : 19.48,
            'Oaxaca' : 17,
            'Quintana Roo' : 19.67,
            'Colima' : 19.17,
            'San luis Potosí' : 22.50,
            'Nuevo León' : 26.24,
            'Querétaro' : 20.85,
            'Tamaulipas' : 23.99,
            'Puebla' : 18.83,
            'Yucatán' : 20.68,
            'Morelos' : 18.75,
            'Guanajuato' : 20.99,
            'Guerrero' : 17.67,
            'Tabasco' : 17.95, 
            'Hidalgo' : 20.5,
            'Veracruz' : 19.33,
            'Chihuahua' : 28.5,
            'Aguascalientes' : 22,
            'Sonora' : 29.33, 
            'Michoacán' : 19.21,
            'Baja California Norte' : 22.16,
            'Baja California Sur' : 25.58,
            'Coahuila' : 27.33,
            'Durango' : 24.83, 
            'Sinaloa' : 25,
            'Chiapas' : 16.5,
            'Nayarit' : 22,
            'Tlaxcala' : 19.41,
            'Campeche' : 19.34,
            'Zacatecas' : 23
        } 


longitud = { 'Distrito Federal' : -99.15, 
             'Jalisco' : -103.67,
             'Edo. de México' : -99.69,
             'Oaxaca' : -96.5,
             'Quintana Roo' : -88.50,
             'Colima' : -104,
             'San luis Potosí' : -100.50,
             'Nuevo León' : -99.89,
             'Querétaro' : -99.84,
             'Tamaulipas' : -98.70,
             'Puebla' : -98, 
             'Yucatán' : -88.88,
             'Morelos' : -99,
             'Guanajuato' : -101, 
             'Guerrero' : -100,
             'Tabasco' : -92.48,
             'Hidalgo' : -99, 
             'Veracruz' : -96.67,
             'Chihuahua' : -106,
             'Aguascalientes' : -102.5,
             'Sonora' : -110.67,
             'Michoacán' : -101.88,
             'Baja California Norte' : -100.96,
             'Baja California Sur' : -111.57, 
             'Coahuila' : -102,
             'Durango' : -104.83, 
             'Sinaloa' : -107.5, 
             'Chiapas' : -92.5,
             'Nayarit' : -105,
             'Tlaxcala' : -98.17,
             'Campeche' : -89.94, 
             'Zacatecas' : -103
        } 

In [11]:
for p in provincias:
    dfinicial.loc[ dfinicial['provincia'] == p, 'lat'] = latitud[p]
    dfinicial.loc[ dfinicial['provincia'] == p, 'lng'] = longitud[p]
    dftest.loc[ dftest['provincia'] == p, 'lat'] = latitud[p]
    dftest.loc[ dftest['provincia'] == p, 'lng'] = longitud[p]

In [12]:
dfinicial[ dfinicial['lat'] == 0 ] #NO TIENE QUE MOSTRAR NADA

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio


In [13]:
dfinicial[ dfinicial['lng'] == 0 ] #NO TIENE QUE MOSTRAR NADA

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio


In [14]:
dfinicial = dfinicial.drop( columns = ['ciudad', 'provincia'] )
dftest = dftest.drop( columns = ['ciudad', 'provincia'] )

### Se pasa fecha a datetime y se agrega mes y año

In [15]:
dfinicial['dia'] = dfinicial['fecha'].dt.day
dfinicial['mes'] = dfinicial['fecha'].dt.month
dfinicial['anio'] = dfinicial['fecha'].dt.year
dfinicial = dfinicial.drop( columns = ['fecha'] )

dftest['dia'] = dftest['fecha'].dt.day
dftest['mes'] = dftest['fecha'].dt.month
dftest['anio'] = dftest['fecha'].dt.year
dftest = dftest.drop( columns = ['fecha'] )

### Para los metros cuadrados (ambos), se rellena los nan con el promedio anual por tipo de propiedad

In [16]:
avgs_m2_cub = dfinicial.groupby(['tipodepropiedad', 'anio']).mean()['metroscubiertos'].reset_index()
avgs_m2_tot = dfinicial.groupby(['tipodepropiedad', 'anio']).mean()['metrostotales'].reset_index()

avg_gen_cub = dfinicial.groupby('tipodepropiedad').mean()['metroscubiertos'].to_list()[0]
avg_gen_tot = dfinicial.groupby('tipodepropiedad').mean()['metrostotales'].to_list()[0]

#Se rellena con el promedio total para los casos donde para ningún año la propiedad tiene un valor de metros
avgs_m2_cub['metroscubiertos'] = avgs_m2_cub['metroscubiertos'].fillna( avg_gen_cub )
avgs_m2_tot['metrostotales']  = avgs_m2_tot['metrostotales'].fillna( avg_gen_tot )

In [17]:
avgs_m2_cub2 = dftest.groupby(['tipodepropiedad', 'anio']).mean()['metroscubiertos'].reset_index()
avgs_m2_tot2 = dftest.groupby(['tipodepropiedad', 'anio']).mean()['metrostotales'].reset_index()

avg_gen_cub2 = dftest.groupby('tipodepropiedad').mean()['metroscubiertos'].to_list()[0]
avg_gen_tot2 = dftest.groupby('tipodepropiedad').mean()['metrostotales'].to_list()[0]

#Se rellena con el promedio total para los casos donde para ningún año la propiedad tiene un valor de metros
avgs_m2_cub2['metroscubiertos'] = avgs_m2_cub2['metroscubiertos'].fillna( avg_gen_cub2 )
avgs_m2_tot2['metrostotales']  = avgs_m2_tot2['metrostotales'].fillna( avg_gen_tot2 )

In [18]:
propiedades = ['Apartamento',
'Bodega comercial',    
'Casa',
'Casa en condominio',
'Casa uso de suelo',
'Departamento Compartido',
'Duplex',
'Edificio', 
'Garage',
'Hospedaje',
'Huerta',
'Inmuebles productivos urbanos',
'Local Comercial',
'Local en centro comercial',      
'Lote',
'Nave industrial',
'Oficina comercial',
'Otros',
'Quinta Vacacional',
'Rancho',
'Terreno',
'Terreno comercial',
'Terreno industrial',
'Villa']

anios = [2012, 2013, 2014, 2015, 2016]

In [19]:
for p in propiedades:
    for a in anios:     
        try:
            dfinicial.loc[ ( ( dfinicial['tipodepropiedad'] == p ) &                 \
                           ( dfinicial['anio'] == a ) ), 'metroscubiertos' ] \
                            =                                  \
                            dfinicial[ ( ( dfinicial['tipodepropiedad'] == p ) &     \
                           ( dfinicial['anio'] == a ) ) ] \
                            ['metroscubiertos'].fillna(                            \
                            avgs_m2_cub[ ( avgs_m2_cub['tipodepropiedad'] == p ) & \
                            ( avgs_m2_cub['anio'] == a ) ]['metroscubiertos'].to_list()[0]
                            )

            dfinicial.loc[ ( ( dfinicial['tipodepropiedad'] == p ) &                 \
                           ( dfinicial['anio'] == a ) ), 'metrostotales' ] \
                            =                                  \
                            dfinicial[ ( ( dfinicial['tipodepropiedad'] == p ) &     \
                           ( dfinicial['anio'] == a ) ) ] \
                            ['metrostotales'].fillna(                            \
                            avgs_m2_tot[ ( avgs_m2_cub['tipodepropiedad'] == p ) & \
                            ( avgs_m2_tot['anio'] == a ) ]['metrostotales'].to_list()[0]
                            )        
            
        except: #Hay casos de (propiedad, anio) que no existen en el dataset
            continue

In [20]:
for p in propiedades:
    for a in anios:     
        try:
            dftest.loc[ ( ( dftest['tipodepropiedad'] == p ) &                 \
                           ( dftest['anio'] == a ) ), 'metroscubiertos' ] \
                            =                                  \
                            dftest[ ( ( dftest['tipodepropiedad'] == p ) &     \
                           ( dftest['anio'] == a ) ) ] \
                            ['metroscubiertos'].fillna(                            \
                            avgs_m2_cub2[ ( avgs_m2_cub2['tipodepropiedad'] == p ) & \
                            ( avgs_m2_cub2['anio'] == a ) ]['metroscubiertos'].to_list()[0]
                            )

            dftest.loc[ ( ( dftest['tipodepropiedad'] == p ) &                 \
                           ( dftest['anio'] == a ) ), 'metrostotales' ] \
                            =                                  \
                            dftest[ ( ( dftest['tipodepropiedad'] == p ) &     \
                           ( dftest['anio'] == a ) ) ] \
                            ['metrostotales'].fillna(                            \
                            avgs_m2_tot2[ ( avgs_m2_cub2['tipodepropiedad'] == p ) & \
                            ( avgs_m2_tot2['anio'] == a ) ]['metrostotales'].to_list()[0]
                            )        
            
        except: #Hay casos de (propiedad, anio) que no existen en el dataset
            continue

### El resto de los features básicos (antiguedad, habitaciones, garages y banos) son discretos, por lo que se los reemplaza con la moda por tipo de propiedad y por año


In [21]:
avgs_banos = dfinicial.groupby(['tipodepropiedad', 'anio'])['banos'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index() # La moda para cada propiedad y anio

avgs_banos['banos'] = avgs_banos['banos'].fillna(0) # los nan tienen 0 baños

# Las "listas vacías" tienen 0 baños
avgs_banos['banos'] = avgs_banos['banos'].astype(str)
avgs_banos.loc[ avgs_banos['banos'] == '[]' , 'banos'] = '0'

# Dejar en formato int
avgs_banos['banos'] = avgs_banos['banos'].astype(np.float).astype(np.uint8)

#####################

avgs_garages = dfinicial.groupby(['tipodepropiedad', 'anio'])['garages'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index() 

avgs_garages['garages'] = avgs_garages['garages'].fillna(0) 

avgs_garages['garages'] = avgs_garages['garages'].astype(str)
avgs_garages.loc[ avgs_garages['garages'] == '[]', 'garages'] = '0'

avgs_garages['garages'] = avgs_garages['garages'].astype(np.float).astype(np.uint8)


###################


avgs_habitaciones = dfinicial.groupby(['tipodepropiedad', 'anio'])['habitaciones'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index() 

avgs_habitaciones['habitaciones'] = avgs_habitaciones['habitaciones'].fillna(0) 

avgs_habitaciones['habitaciones'] = avgs_habitaciones['habitaciones'].astype(str)
avgs_habitaciones.loc[ avgs_habitaciones['habitaciones'] == '[]', 'habitaciones'] = '0'

avgs_habitaciones['habitaciones'] = avgs_habitaciones['habitaciones'].astype(np.float).astype(np.uint8)

#################


avgs_antiguedad = dfinicial.groupby(['tipodepropiedad', 'anio'])['antiguedad'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index()

avgs_antiguedad['antiguedad'] = avgs_antiguedad['antiguedad'].fillna(0)

avgs_antiguedad['antiguedad'] = avgs_antiguedad['antiguedad'].astype(str)
avgs_antiguedad.loc[ avgs_antiguedad['antiguedad'] == '[]', 'antiguedad'] = '0'

avgs_antiguedad['antiguedad'] = avgs_antiguedad['antiguedad'].astype(np.float).astype(np.uint16)

##################

#Se completa con el promedio total para los casos donde para ningún año la propiedad tiene un valor
avg_ban = dfinicial.groupby('tipodepropiedad').mean()['banos'].to_list()[0]
avg_gar_tot = dfinicial.groupby('tipodepropiedad').mean()['garages'].to_list()[0]
avg_hab = dfinicial.groupby('tipodepropiedad').mean()['habitaciones'].to_list()[0]
avg_ants = dfinicial.groupby('tipodepropiedad').mean()['antiguedad'].to_list()[0]

avgs_banos['banos'] = avgs_banos['banos'].fillna( avg_ban )
avgs_garages['garages']  = avgs_garages['garages'].fillna( avg_gar_tot )
avgs_habitaciones['habitaciones'] = avgs_habitaciones['habitaciones'].fillna( avg_hab )
avgs_antiguedad['antiguedad']  = avgs_antiguedad['antiguedad'].fillna( avg_ants )



In [22]:
avgs_banos2 = dftest.groupby(['tipodepropiedad', 'anio'])['banos'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index() # La moda para cada propiedad y anio

avgs_banos2['banos'] = avgs_banos2['banos'].fillna(0) # los nan tienen 0 baños

# Las "listas vacías" tienen 0 baños
avgs_banos2['banos'] = avgs_banos2['banos'].astype(str)
avgs_banos2.loc[ avgs_banos2['banos'] == '[]' , 'banos'] = '0'

# Dejar en formato int
avgs_banos2['banos'] = avgs_banos2['banos'].astype(np.float).astype(np.uint8)

#####################

avgs_garages2 = dftest.groupby(['tipodepropiedad', 'anio'])['garages'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index() 

avgs_garages2['garages'] = avgs_garages2['garages'].fillna(0) 

avgs_garages2['garages'] = avgs_garages2['garages'].astype(str)
avgs_garages2.loc[ avgs_garages2['garages'] == '[]', 'garages'] = '0'

avgs_garages2['garages'] = avgs_garages2['garages'].astype(np.float).astype(np.uint8)


###################


avgs_habitaciones2 = dftest.groupby(['tipodepropiedad', 'anio'])['habitaciones'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index() 

avgs_habitaciones2['habitaciones'] = avgs_habitaciones2['habitaciones'].fillna(0) 

avgs_habitaciones2['habitaciones'] = avgs_habitaciones2['habitaciones'].astype(str)
avgs_habitaciones2.loc[ avgs_habitaciones2['habitaciones'] == '[]', 'habitaciones'] = '0'

avgs_habitaciones2['habitaciones'] = avgs_habitaciones2['habitaciones'].astype(np.float).astype(np.uint8)

#################


avgs_antiguedad2 = dftest.groupby(['tipodepropiedad', 'anio'])['antiguedad'].agg( lambda x: x.mode()[0] \
            if ( len( x.mode() ) > 0 ) else x.mode() ).reset_index()

avgs_antiguedad2['antiguedad'] = avgs_antiguedad2['antiguedad'].fillna(0)

avgs_antiguedad2['antiguedad'] = avgs_antiguedad2['antiguedad'].astype(str)
avgs_antiguedad2.loc[ avgs_antiguedad2['antiguedad'] == '[]', 'antiguedad'] = '0'

avgs_antiguedad2['antiguedad'] = avgs_antiguedad2['antiguedad'].astype(np.float).astype(np.uint16)

##################

#Se completa con el promedio total para los casos donde para ningún año la propiedad tiene un valor
avg_ban2 = dftest.groupby('tipodepropiedad').mean()['banos'].to_list()[0]
avg_gar_tot2 = dftest.groupby('tipodepropiedad').mean()['garages'].to_list()[0]
avg_hab2 = dftest.groupby('tipodepropiedad').mean()['habitaciones'].to_list()[0]
avg_ants2 = dftest.groupby('tipodepropiedad').mean()['antiguedad'].to_list()[0]

avgs_banos2['banos'] = avgs_banos2['banos'].fillna( avg_ban2 )
avgs_garages2['garages']  = avgs_garages2['garages'].fillna( avg_gar_tot2 )
avgs_habitaciones2['habitaciones'] = avgs_habitaciones2['habitaciones'].fillna( avg_hab2 )
avgs_antiguedad2['antiguedad']  = avgs_antiguedad2['antiguedad'].fillna( avg_ants2 )



In [23]:
for p in propiedades:
    for a in anios:     
        try:
            dfinicial.loc[ ( ( dfinicial['tipodepropiedad'] == p ) &                 \
                           ( dfinicial['anio'] == a ) ), 'banos' ] \
                            =                                  \
                            dfinicial[ ( ( dfinicial['tipodepropiedad'] == p ) &     \
                           ( dfinicial['anio'] == a ) ) ] \
                            ['banos'].fillna(                            \
                            avgs_banos[ ( avgs_banos['tipodepropiedad'] == p ) & \
                            ( avgs_banos['anio'] == a ) ]['banos'].to_list()[0]
                            )
    
            dfinicial.loc[ ( ( dfinicial['tipodepropiedad'] == p ) &                 \
                           ( dfinicial['anio'] == a ) ), 'garages' ] \
                            =                                  \
                            dfinicial[ ( ( dfinicial['tipodepropiedad'] == p ) &     \
                           ( dfinicial['anio'] == a ) ) ] \
                            ['garages'].fillna(                            \
                            avgs_garages[ ( avgs_garages['tipodepropiedad'] == p ) & \
                            ( avgs_garages['anio'] == a ) ]['garages'].to_list()[0]
                            )
      
    
            dfinicial.loc[ ( ( dfinicial['tipodepropiedad'] == p ) &                 \
                           ( dfinicial['anio'] == a ) ), 'antiguedad' ] \
                            =                                  \
                            dfinicial[ ( ( dfinicial['tipodepropiedad'] == p ) &     \
                           ( dfinicial['anio'] == a ) ) ] \
                            ['antiguedad'].fillna(                            \
                            avgs_antiguedad[ ( avgs_antiguedad['tipodepropiedad'] == p ) & \
                            ( avgs_antiguedad['anio'] == a ) ]['antiguedad'].to_list()[0]
                            )
      
            dfinicial.loc[ ( ( dfinicial['tipodepropiedad'] == p ) &                 \
                           ( dfinicial['anio'] == a ) ), 'habitaciones' ] \
                            =                                  \
                            dfinicial[ ( ( dfinicial['tipodepropiedad'] == p ) &     \
                           ( dfinicial['anio'] == a ) ) ] \
                            ['habitaciones'].fillna(                            \
                            avgs_habitaciones[ ( avgs_habitaciones['tipodepropiedad'] == p ) & \
                            ( avgs_habitaciones['anio'] == a ) ]['habitaciones'].to_list()[0]
                            )
      
            
        except: 
            print(p)
            print(a)

Duplex
2012
Duplex
2013
Duplex
2014
Garage
2012
Garage
2013
Garage
2014
Garage
2016
Hospedaje
2012
Hospedaje
2013
Hospedaje
2014
Hospedaje
2015
Lote
2012
Lote
2013
Lote
2015
Otros
2012
Otros
2013
Terreno industrial
2012
Terreno industrial
2013


In [24]:
for p in propiedades:
    for a in anios:     
        try:
            dftest.loc[ ( ( dftest['tipodepropiedad'] == p ) &                 \
                           ( dftest['anio'] == a ) ), 'banos' ] \
                            =                                  \
                            dftest[ ( ( dftest['tipodepropiedad'] == p ) &     \
                           ( dftest['anio'] == a ) ) ] \
                            ['banos'].fillna(                            \
                            avgs_banos2[ ( avgs_banos2['tipodepropiedad'] == p ) & \
                            ( avgs_banos2['anio'] == a ) ]['banos'].to_list()[0]
                            )
    
            dftest.loc[ ( ( dftest['tipodepropiedad'] == p ) &                 \
                           ( dftest['anio'] == a ) ), 'garages' ] \
                            =                                  \
                            dftest[ ( ( dftest['tipodepropiedad'] == p ) &     \
                           ( dftest['anio'] == a ) ) ] \
                            ['garages'].fillna(                            \
                            avgs_garages2[ ( avgs_garages2['tipodepropiedad'] == p ) & \
                            ( avgs_garages2['anio'] == a ) ]['garages'].to_list()[0]
                            )
      
    
            dftest.loc[ ( ( dftest['tipodepropiedad'] == p ) &                 \
                           ( dftest['anio'] == a ) ), 'antiguedad' ] \
                            =                                  \
                            dftest[ ( ( dftest['tipodepropiedad'] == p ) &     \
                           ( dftest['anio'] == a ) ) ] \
                            ['antiguedad'].fillna(                            \
                            avgs_antiguedad2[ ( avgs_antiguedad2['tipodepropiedad'] == p ) & \
                            ( avgs_antiguedad2['anio'] == a ) ]['antiguedad'].to_list()[0]
                            )
      
            dftest.loc[ ( ( dftest['tipodepropiedad'] == p ) &                 \
                           ( dftest['anio'] == a ) ), 'habitaciones' ] \
                            =                                  \
                            dftest[ ( ( dftest['tipodepropiedad'] == p ) &     \
                           ( dftest['anio'] == a ) ) ] \
                            ['habitaciones'].fillna(                            \
                            avgs_habitaciones2[ ( avgs_habitaciones2['tipodepropiedad'] == p ) & \
                            ( avgs_habitaciones2['anio'] == a ) ]['habitaciones'].to_list()[0]
                            )
      
            
        except: 
            print(p)
            print(a)

Duplex
2012
Duplex
2013
Duplex
2014
Garage
2012
Garage
2013
Garage
2014
Garage
2015
Garage
2016
Hospedaje
2012
Hospedaje
2013
Hospedaje
2014
Hospedaje
2015
Hospedaje
2016
Huerta
2012
Huerta
2014
Lote
2012
Lote
2013
Lote
2015
Lote
2016
Otros
2012
Otros
2013
Terreno industrial
2012
Terreno industrial
2013
Terreno industrial
2014
Terreno industrial
2015


### Se adapta el tipo de propiedad a un ranking (según el precio promedio por vivienda) y se adecúan los tipos de datos a formato int

In [25]:
props = {'Apartamento' : 14,
'Bodega comercial' : 15,    
'Casa' : 18,
'Casa en condominio' : 11,
'Casa uso de suelo' : 5,
'Departamento Compartido' : 16,
'Duplex' : 22,
'Edificio' : 1, 
'Garage' : 24,
'Hospedaje' : 10,
'Huerta' : 7,
'Inmuebles productivos urbanos' : 3,
'Local Comercial' : 19,
'Local en centro comercial' : 20,      
'Lote' : 23,
'Nave industrial' : 4,
'Oficina comercial' : 9,
'Otros' : 12,
'Quinta Vacacional' : 8,
'Rancho' : 2,
'Terreno' : 21,
'Terreno comercial' : 17,
'Terreno industrial' : 6,
'Villa' : 13}

for p in props:
    dfinicial.loc[ dfinicial['tipodepropiedad'] == p, 'tipodepropiedad'] = props[p]   
    dftest.loc[ dftest['tipodepropiedad'] == p, 'tipodepropiedad'] = props[p]

In [26]:
dfinicial['tipodepropiedad'] = dfinicial['tipodepropiedad'].astype(np.uint8)
dfinicial['garages'] = dfinicial['garages'].astype(np.uint8)
dfinicial['antiguedad'] = dfinicial['antiguedad'].astype(np.uint8)
dfinicial['banos'] = dfinicial['banos'].astype(np.uint8)
dfinicial['habitaciones'] = dfinicial['habitaciones'].astype(np.uint8)

dfinicial['metroscubiertos'] = dfinicial['metroscubiertos'].astype(np.uint16)
dfinicial['metrostotales'] = dfinicial['metrostotales'].astype(np.uint16)
dfinicial['lat'] = dfinicial['lat'].astype(np.int16)
dfinicial['lng'] = dfinicial['lng'].astype(np.int16)

dfinicial['dia'] = dfinicial['dia'].astype(np.uint8)
dfinicial['mes'] = dfinicial['mes'].astype(np.uint8)
dfinicial['anio'] = dfinicial['anio'].astype(np.uint16)

In [27]:
dftest['tipodepropiedad'] = dftest['tipodepropiedad'].astype(np.uint8)
dftest['garages'] = dftest['garages'].astype(np.uint8)
dftest['antiguedad'] = dftest['antiguedad'].astype(np.uint8)
dftest['banos'] = dftest['banos'].astype(np.uint8)
dftest['habitaciones'] = dftest['habitaciones'].astype(np.uint8)

dftest['metroscubiertos'] = dftest['metroscubiertos'].astype(np.uint16)
dftest['metrostotales'] = dftest['metrostotales'].astype(np.uint16)
dftest['lat'] = dftest['lat'].astype(np.int16)
dftest['lng'] = dftest['lng'].astype(np.int16)

dftest['dia'] = dftest['dia'].astype(np.uint8)
dftest['mes'] = dftest['mes'].astype(np.uint8)
dftest['anio'] = dftest['anio'].astype(np.uint16)

In [28]:
dfinicial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239600 entries, 0 to 239999
Data columns (total 18 columns):
tipodepropiedad               239600 non-null uint8
antiguedad                    239600 non-null uint8
habitaciones                  239600 non-null uint8
garages                       239600 non-null uint8
banos                         239600 non-null uint8
metroscubiertos               239600 non-null uint16
metrostotales                 239600 non-null uint16
lat                           239600 non-null int16
lng                           239600 non-null int16
gimnasio                      239600 non-null uint8
usosmultiples                 239600 non-null uint8
piscina                       239600 non-null uint8
escuelascercanas              239600 non-null uint8
centroscomercialescercanos    239600 non-null uint8
precio                        239600 non-null uint32
dia                           239600 non-null uint8
mes                           239600 non-null uint8
an

# 3. Feature Engineering

In [58]:
'''dfinicial1 = dfinicial.drop( columns = [ 'dia'
                           ])                                        
 
dftest1 = dftest.drop( columns = [      'dia'
                           ])                                        
 '''

"dfinicial1 = dfinicial.drop( columns = [ 'dia'\n                           ])                                        \n \ndftest1 = dftest.drop( columns = [      'dia'\n                           ])                                        \n "

### 4. Algoritmos Probados

In [56]:
def generar_subida(DFINICIAL, metodo, DFTESTEAR):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
    X, y = DFINICIAL.drop( columns = ['precio'] ), dfinicial["precio"]

    reg = metodo()
    reg.fit(X,y)
    
    subida = pd.DataFrame( reg.predict(DFTESTEAR) , columns=["target"])
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    # Heurística: si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    
    # Heurística 2: si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    subida.to_csv(f"submits/prueba_esteban.csv", index = False)
    
    return subida

### 4.1. Linear Regression

In [107]:
dfinicial1 = dfinicial.drop( columns = [ 'anio', 'lat', 'metrostotales'
                           ])                                        
 
dftest1 = dftest.drop( columns = [  'anio', 'lat', 'metrostotales'
                           ])                                        

subida = generar_subida( dfinicial ,  LinearRegression, dftest)

xg = pd.read_csv('submits/submit_xg_prueba.csv')
dftot = subida.merge(xg, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )


# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )

817305.0